In [ ]:
import pandas as pd
import time
import numpy as np
from helper import *
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import timedelta, date, datetime

In [ ]:
def get_market(df, tic):
    df_res = df[df['tic'] == tic].copy()
    df_res['ratio'] = convert_prices(df_res[["close"]])
    df_res['diff'] = df_res['close'] - df_res['open']
    df_res.loc[df_res['diff']>=0, 'color'] = 'green'
    df_res.loc[df_res['diff']<0, 'color'] = 'red'
    return df_res
    
def plot_line(df, title, split_dates=[]):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df['date'],y=df['open'],name='Price'),secondary_y=False)
    for i, split in enumerate(split_dates):
        color = 'black' if i == 0 else 'red' if i == 1 else 'blue'
        fig.add_vline(x=split, line_width=2, line_color=color, opacity=0.5)
    # fig.add_trace(go.Scatter(x=df['date'],y=df['ratio'],name='Ratio'),secondary_y=True)
    fig.add_trace(go.Bar(x=df['date'],y=df['volume'],opacity=0.5, name='volume'),secondary_y=True)
    # fig.update_yaxes(visible=False, secondary_y=True)
    fig.update_layout(title_text=title, height=600)
    fig.show()
    fig.write_html(f"plots/{title.replace(' ','_')}.html")
    # fig.savefig(f"data_exploration/{title.replace(' ', '_')}.png")

def plot_candle(df, title):
    fig3 = make_subplots(specs=[[{"secondary_y": True}]])
    fig3.add_trace(go.Candlestick(x=df['date'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='price'))
    fig3.add_trace(go.Scatter(x=df['date'],y=df['close'].rolling(window=20).mean(),marker_color='blue',name='20 Day MA'))
    fig3.add_trace(go.Bar(x=df['date'], y=df['volume'], name='Volume', opacity=0.5, marker={'color':df['color']}),secondary_y=True)
    # fig3.update_yaxes(range=[0,25000000000],secondary_y=True)
    fig3.update_layout(xaxis_rangeslider_visible=False, title_text=title, height=600)
    fig3.show()
# fig.update_layout(height=600, width=800, )



In [ ]:
df = get_data('1d')

In [ ]:
start_train = datetime.strptime("11.05.2021", "%d.%m.%Y") 
end_train = start_train + timedelta(days=378, hours=23, minutes=59, seconds=59)
split_dates = []
for i in range(7):
    split_dates.append(end_train)
    end_train += timedelta(days=28)

In [ ]:
# ticker = 'BTCUSDT'
# btc = get_market(df, ticker)
# plot_line(btc, ticker, split_dates)

In [ ]:
split_dates

In [ ]:
for ticker in get_ticker_name_list(df):
    market_data = get_market(df, ticker)
    plot_line(market_data, f"{ticker} 7d", split_dates)

In [ ]:
len(split_dates)

In [ ]:
# df1 = df[df['tic']=='BTCUSDT']
# df2 = df[df['tic']=='DOGEUSDT']

In [ ]:
colors = ["#FF0000", "#18E3FF", "#FFA90C", "#D9E501", "#0A2DC2", "#05A41B", "#878787", "#935948", "#b372fa", "#FC12DD"]

def plot_performance(df, split_dates=[], title='Complete'):
    ratios = convert_prices(get_close_list(df))
    ratios = ratios.cumprod()
    datex = ratios.index

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    for idx, ticker in enumerate(ratios.columns):
        fig.add_trace(go.Scatter(x=datex,y=ratios[ticker],name=ticker, marker_color=colors[idx]),secondary_y=False)

    for i, split in enumerate(split_dates):
        if len(split_dates) > 10:
            color = 'black' if  i == 0 else 'red' if i == 1 else 'blue'
        else:
            color = 'black'
        fig.add_vline(x=split, line_width=1, line_color=color, opacity=1)
        
    fig.update_layout(height=600) # title_text=f'{title} Dataset', 
    fig.update_yaxes(type="log")
    fig.show()
    fig.write_html(f"plots_performance/{title.replace(' ','_')}.html")
    fig.write_image(f"plots_performance/{title.replace(' ','_')}.png", width=1200)

In [ ]:
train_set = df[(df['date'] <= split_dates[0])]

In [ ]:
closetrain = get_close_list(train_set)
ctrain = convert_prices(closetrain).cumprod()
closeall = get_close_list(df)
call = convert_prices(closeall).cumprod()
closetest = get_close_list(df_test)
ctest = convert_prices(closetest).cumprod()

In [ ]:
print("train")
closetrain.describe()

In [ ]:
closetrain

In [ ]:
train_set[train_set['tic'] == "AXSUSDT"]

In [ ]:
len(split_dates)

In [ ]:
plot_performance(df, split_dates)

In [ ]:
df_test = df[df['date'] >= str(split_dates[0])].copy()
plot_performance(df_test, split_dates[1:], 'Trading (Test)')